In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['font.size'] = 15
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['figure.figsize'] = 16,8

import warnings
warnings.filterwarnings('ignore')


import re
from konlpy.tag import Okt, Mecab
from hanspell import spell_checker
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize

import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HOME\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## 데이터 불러오기

In [14]:
tag_df = pd.read_json('data/tag_konlpy.json')
title_to_tag_df = pd.read_json('data/title_to_tag_konlpy.json')

In [16]:
tag_df.reset_index(inplace=True)
tag_df

,index,tags__
0,0,[락]
1,1,"[추억, 회상]"
2,2,"[카페, 잔잔]"
3,3,"[연말, 눈, 오다, 날, 캐럴, 분위기, 따, 크리스마스, 캐럴, 겨울, 노래, ..."
4,4,[댄스]
...,...,...
115066,115066,"[록, 메탈, 밴드, 사운드, 록, 락, 메탈, 메탈, 락, extreme]"
115067,115067,[일렉]
115068,115068,"[담시, 가족, 눈물, 그리움, 주인공, 이야기, 사랑, 친구]"
115069,115069,"[잔잔, 버스, 퇴근, 버스, pop, 풍경, 퇴근길]"


In [17]:
title_to_tag_df.reset_index(inplace=True)
title_to_tag_df

,index,plylist_title__
0,0,"[여행, 음악]"
1,1,[요즘]
2,2,"[편하, 잔잔, 곡]"
3,3,"[크리스마스, 분위기, 흠뻑, 취하]"
4,4,"[추억, 노래]"
...,...,...
115066,115066,"[metal, e, sm, 2]"
115067,115067,"[빠르, 리스, 위, 따끈따끈, 최신, 인기, edm, 모음]"
115068,115068,"[1, 눈물, 가리, 이야기]"
115069,115069,"[퇴근, 버스, 편하, 하루, 마무리, pop]"


## 데이터 합쳐서 새로운 태그로 만들기

In [18]:
total = pd.merge(tag_df,title_to_tag_df, on='index', how='left')
total['new_tags'] = total['tags__'] + total['plylist_title__']
total.drop('index',axis=1,inplace=True)
total

,tags__,plylist_title__,new_tags
0,[락],"[여행, 음악]","[락, 여행, 음악]"
1,"[추억, 회상]",[요즘],"[추억, 회상, 요즘]"
2,"[카페, 잔잔]","[편하, 잔잔, 곡]","[카페, 잔잔, 편하, 잔잔, 곡]"
3,"[연말, 눈, 오다, 날, 캐럴, 분위기, 따, 크리스마스, 캐럴, 겨울, 노래, ...","[크리스마스, 분위기, 흠뻑, 취하]","[연말, 눈, 오다, 날, 캐럴, 분위기, 따, 크리스마스, 캐럴, 겨울, 노래, ..."
4,[댄스],"[추억, 노래]","[댄스, 추억, 노래]"
...,...,...,...
115066,"[록, 메탈, 밴드, 사운드, 록, 락, 메탈, 메탈, 락, extreme]","[metal, e, sm, 2]","[록, 메탈, 밴드, 사운드, 록, 락, 메탈, 메탈, 락, extreme, met..."
115067,[일렉],"[빠르, 리스, 위, 따끈따끈, 최신, 인기, edm, 모음]","[일렉, 빠르, 리스, 위, 따끈따끈, 최신, 인기, edm, 모음]"
115068,"[담시, 가족, 눈물, 그리움, 주인공, 이야기, 사랑, 친구]","[1, 눈물, 가리, 이야기]","[담시, 가족, 눈물, 그리움, 주인공, 이야기, 사랑, 친구, 1, 눈물, 가리,..."
115069,"[잔잔, 버스, 퇴근, 버스, pop, 풍경, 퇴근길]","[퇴근, 버스, 편하, 하루, 마무리, pop]","[잔잔, 버스, 퇴근, 버스, pop, 풍경, 퇴근길, 퇴근, 버스, 편하, 하루,..."


### 새로운 태그 저장

In [20]:
total.to_json('data/totla_new_tag.json', orient = 'records')

### 새로운 태그 단어 빈도수 확인

In [21]:
unnest = np.dstack(
    (np.repeat(total.index.values,list(map(len,total.new_tags))),
     np.concatenate(total.new_tags.values))
         )

unnest_df = pd.DataFrame(unnest[0],columns = ['id','tagss'])

ddf = pd.DataFrame(unnest_df['tagss'].value_counts())
ddf.reset_index(inplace=True)
ddf.columns = ['tagss','tag_cnt']
ddf

,tagss,tag_cnt
0,노래,24848
1,감성,22993
2,음악,22952
3,듣,17105
4,기분전환,17060
...,...,...
24042,well,1
24043,임산,1
24044,pino,1
24045,paulmauriat,1


In [22]:
ddf.query('tagss == "신나는"')[150:200]

,tagss,tag_cnt


In [23]:
ddf.describe()

,tag_cnt
count,24047.000000
mean,45.073689
std,498.260858
min,1.000000
25%,1.000000
50%,2.000000
75%,7.000000
max,24848.000000


---
---
---
## 필요없는 태그 지우기 

In [33]:
new_tag = total.copy()
new_tag

,tags__,plylist_title__,new_tags
0,[락],"[여행, 음악]","[락, 여행, 음악]"
1,"[추억, 회상]",[요즘],"[추억, 회상, 요즘]"
2,"[카페, 잔잔]","[편하, 잔잔, 곡]","[카페, 잔잔, 편하, 잔잔, 곡]"
3,"[연말, 눈, 오다, 날, 캐럴, 분위기, 따, 크리스마스, 캐럴, 겨울, 노래, ...","[크리스마스, 분위기, 흠뻑, 취하]","[연말, 눈, 오다, 날, 캐럴, 분위기, 따, 크리스마스, 캐럴, 겨울, 노래, ..."
4,[댄스],"[추억, 노래]","[댄스, 추억, 노래]"
...,...,...,...
115066,"[록, 메탈, 밴드, 사운드, 록, 락, 메탈, 메탈, 락, extreme]","[metal, e, sm, 2]","[록, 메탈, 밴드, 사운드, 록, 락, 메탈, 메탈, 락, extreme, met..."
115067,[일렉],"[빠르, 리스, 위, 따끈따끈, 최신, 인기, edm, 모음]","[일렉, 빠르, 리스, 위, 따끈따끈, 최신, 인기, edm, 모음]"
115068,"[담시, 가족, 눈물, 그리움, 주인공, 이야기, 사랑, 친구]","[1, 눈물, 가리, 이야기]","[담시, 가족, 눈물, 그리움, 주인공, 이야기, 사랑, 친구, 1, 눈물, 가리,..."
115069,"[잔잔, 버스, 퇴근, 버스, pop, 풍경, 퇴근길]","[퇴근, 버스, 편하, 하루, 마무리, pop]","[잔잔, 버스, 퇴근, 버스, pop, 풍경, 퇴근길, 퇴근, 버스, 편하, 하루,..."


In [34]:
X = [list(set(x)) for x in new_tag['new_tags']]
new_tag['no_du'] = X
new_tag.drop(['tags__','plylist_title__','new_tags'],axis=1,inplace=True)
new_tag

,no_du
0,"[음악, 락, 여행]"
1,"[요즘, 회상, 추억]"
2,"[잔잔, 편하, 곡, 카페]"
3,"[노래, 따, 캐럴, 분위기, 크리스마스, 겨울, 송, 눈, 흠뻑, 왕국, 취하, ..."
4,"[노래, 댄스, 추억]"
...,...
115066,"[락, 사운드, extreme, metal, 록, 2, 밴드, sm, 메탈, e]"
115067,"[edm, 인기, 모음, 최신, 위, 리스, 따끈따끈, 빠르, 일렉]"
115068,"[눈물, 담시, 가족, 1, 주인공, 그리움, 사랑, 친구, 이야기, 가리]"
115069,"[잔잔, 버스, pop, 마무리, 풍경, 퇴근길, 편하, 하루, 퇴근]"


In [29]:
unnest = np.dstack(
    (np.repeat(new_tag.index.values,list(map(len,new_tag.no_du))),
     np.concatenate(new_tag.no_du.values))
         )

unnest_df = pd.DataFrame(unnest[0],columns = ['id','no_du'])

ddf = pd.DataFrame(unnest_df['no_du'].value_counts())
ddf.reset_index(inplace=True)
ddf.columns = ['no_du','cnt']
ddf

,no_du,cnt
0,노래,21424
1,음악,19166
2,감성,17736
3,기분전환,16700
4,듣,16065
...,...,...
24042,987,1
24043,뚱기,1
24044,anitabaker,1
24045,neworleans,1


### 일정 개수 이하의 태그 지우기 
- 단어 안지운거 
- 1개 지운거
- 2개 지운거 
- 3개 지운거 만들기

### 안지운거

In [66]:
rem = ['음악','모음','노래','곡','날','오다','추천','위','은','대','도','b','r','다','성','송','귀','뮤직','music','차트',
       'vol']

# 단어 제거하기

clean = []

for big_list in new_tag['no_du']:
    temp_list = []
    for word in big_list:
        if word not in rem:
            temp_list.append(word)
    clean.append(temp_list)
            
A0 = []
A0.append(clean)

new_tag0 = pd.DataFrame(A0).T
new_tag0.columns = ['안지움']
new_tag0

,안지움
0,"[락, 여행]"
1,"[요즘, 회상, 추억]"
2,"[잔잔, 편하, 카페]"
3,"[따, 캐럴, 분위기, 크리스마스, 겨울, 눈, 흠뻑, 왕국, 취하, 연말]"
4,"[댄스, 추억]"
...,...
115066,"[락, 사운드, extreme, metal, 록, 2, 밴드, sm, 메탈, e]"
115067,"[edm, 인기, 최신, 리스, 따끈따끈, 빠르, 일렉]"
115068,"[눈물, 담시, 가족, 1, 주인공, 그리움, 사랑, 친구, 이야기, 가리]"
115069,"[잔잔, 버스, pop, 마무리, 풍경, 퇴근길, 편하, 하루, 퇴근]"


In [68]:
unnest = np.dstack(
    (np.repeat(new_tag0.index.values,list(map(len,new_tag0['안지움']))),
     np.concatenate(new_tag0['안지움'].values))
         )

unnest_df = pd.DataFrame(unnest[0],columns = ['id','안지움'])

ddf0 = pd.DataFrame(unnest_df['안지움'].value_counts())
ddf0.reset_index(inplace=True)
ddf0.columns = ['안지움','cnt']
ddf0

,안지움,cnt
0,감성,17736
1,기분전환,16700
2,듣,16065
3,발라드,12181
4,잔잔,11959
...,...,...
24021,mamas,1
24022,backward,1
24023,데렉,1
24024,bonus,1


In [69]:
new_tag0.to_json('data/new_tag/new_tag0.json', orient = 'records')

### 1개 지운거

In [76]:
app = ['음악','모음','노래','곡','날','오다','추천','위','은','대','도','b','r','다','성','송','귀','뮤직','music','차트',
       'vol']
rem1 = ddf.query('cnt <= 1')['no_du'].values
rem1 = np.append(rem1,app)
rem1

# 단어 제거하기

clean_1 = []

for big_list in new_tag['no_du']:
    temp_list = []
    for word in big_list:
        if word not in rem1:
            temp_list.append(word)
    clean_1.append(temp_list)
    
A1 = []
A1.append(clean_1)

new_tag1 = pd.DataFrame(A1).T
new_tag1.columns = ['한개지움']
new_tag1

,한개지움
0,"[락, 여행]"
1,"[요즘, 회상, 추억]"
2,"[잔잔, 편하, 카페]"
3,"[따, 캐럴, 분위기, 크리스마스, 겨울, 눈, 흠뻑, 왕국, 취하, 연말]"
4,"[댄스, 추억]"
...,...
115066,"[락, 사운드, extreme, metal, 록, 2, 밴드, sm, 메탈, e]"
115067,"[edm, 인기, 최신, 리스, 따끈따끈, 빠르, 일렉]"
115068,"[눈물, 담시, 가족, 1, 주인공, 그리움, 사랑, 친구, 이야기, 가리]"
115069,"[잔잔, 버스, pop, 마무리, 풍경, 퇴근길, 편하, 하루, 퇴근]"


In [77]:
unnest = np.dstack(
    (np.repeat(new_tag1.index.values,list(map(len,new_tag1['한개지움']))),
     np.concatenate(new_tag1['한개지움'].values))
         )

unnest_df = pd.DataFrame(unnest[0],columns = ['id','한개지움'])

ddf1 = pd.DataFrame(unnest_df['한개지움'].value_counts())
ddf1.reset_index(inplace=True)
ddf1.columns = ['한개지움','cnt']
ddf1

,한개지움,cnt
0,감성,17736
1,기분전환,16700
2,듣,16065
3,발라드,12181
4,잔잔,11959
...,...,...
13523,메이든,2
13524,ohn,2
13525,plus,2
13526,수의,2


In [78]:
new_tag1.to_json('data/new_tag/new_tag1.json', orient = 'records')

### 2개 지운거

In [79]:
app = ['음악','모음','노래','곡','날','오다','추천','위','은','대','도','b','r','다','성','송','귀','뮤직','music','차트',
       'vol']
rem2 = ddf.query('cnt <= 2')['no_du'].values
rem2 = np.append(rem2,app)
rem2

# 단어 제거하기

clean_2 = []

for big_list in new_tag['no_du']:
    temp_list = []
    for word in big_list:
        if word not in rem2:
            temp_list.append(word)
    clean_2.append(temp_list)
    
A2 = []
A2.append(clean_2)
            
new_tag2 = pd.DataFrame(A2).T
new_tag2.columns = ['두개지움']
new_tag2

,두개지움
0,"[락, 여행]"
1,"[요즘, 회상, 추억]"
2,"[잔잔, 편하, 카페]"
3,"[따, 캐럴, 분위기, 크리스마스, 겨울, 눈, 흠뻑, 왕국, 취하, 연말]"
4,"[댄스, 추억]"
...,...
115066,"[락, 사운드, extreme, metal, 록, 2, 밴드, sm, 메탈, e]"
115067,"[edm, 인기, 최신, 리스, 따끈따끈, 빠르, 일렉]"
115068,"[눈물, 담시, 가족, 1, 주인공, 그리움, 사랑, 친구, 이야기, 가리]"
115069,"[잔잔, 버스, pop, 마무리, 풍경, 퇴근길, 편하, 하루, 퇴근]"


In [80]:
unnest = np.dstack(
    (np.repeat(new_tag2.index.values,list(map(len,new_tag2['두개지움']))),
     np.concatenate(new_tag2['두개지움'].values))
         )

unnest_df = pd.DataFrame(unnest[0],columns = ['id','두개지움'])

ddf2 = pd.DataFrame(unnest_df['두개지움'].value_counts())
ddf2.reset_index(inplace=True)
ddf2.columns = ['두개지움','cnt']
ddf2

,두개지움,cnt
0,감성,17736
1,기분전환,16700
2,듣,16065
3,발라드,12181
4,잔잔,11959
...,...,...
10143,보임,3
10144,금물,3
10145,산레모,3
10146,래시,3


In [81]:
new_tag2.to_json('data/new_tag/new_tag2.json', orient = 'records')

### 3개 지운거

In [82]:
app = ['음악','모음','노래','곡','날','오다','추천','위','은','대','도','b','r','다','성','송','귀','뮤직','music','차트',
       'vol']
rem3 = ddf.query('cnt <= 3')['no_du'].values
rem3 = np.append(rem3,app)
rem3

# 단어 제거하기

clean_3 = []

for big_list in new_tag['no_du']:
    temp_list = []
    for word in big_list:
        if word not in rem3:
            temp_list.append(word)
    clean_3.append(temp_list)
            
A3 = []
A3.append(clean_3)
            
new_tag3 = pd.DataFrame(A3).T
new_tag3.columns = ['세개지움']
new_tag3

,세개지움
0,"[락, 여행]"
1,"[요즘, 회상, 추억]"
2,"[잔잔, 편하, 카페]"
3,"[따, 캐럴, 분위기, 크리스마스, 겨울, 눈, 흠뻑, 왕국, 취하, 연말]"
4,"[댄스, 추억]"
...,...
115066,"[락, 사운드, metal, 록, 2, 밴드, sm, 메탈, e]"
115067,"[edm, 인기, 최신, 리스, 따끈따끈, 빠르, 일렉]"
115068,"[눈물, 가족, 1, 주인공, 그리움, 사랑, 친구, 이야기, 가리]"
115069,"[잔잔, 버스, pop, 마무리, 풍경, 퇴근길, 편하, 하루, 퇴근]"


In [83]:
unnest = np.dstack(
    (np.repeat(new_tag3.index.values,list(map(len,new_tag3['세개지움']))),
     np.concatenate(new_tag3['세개지움'].values))
         )

unnest_df = pd.DataFrame(unnest[0],columns = ['id','세개지움'])

ddf3 = pd.DataFrame(unnest_df['세개지움'].value_counts())
ddf3.reset_index(inplace=True)
ddf3.columns = ['세개지움','cnt']
ddf3

,세개지움,cnt
0,감성,17736
1,기분전환,16700
2,듣,16065
3,발라드,12181
4,잔잔,11959
...,...,...
8259,joe,4
8260,침울,4
8261,성운,4
8262,후니,4


In [84]:
new_tag3.to_json('data/new_tag/new_tag3.json', orient = 'records')

---
---
---

---
---
---

In [87]:
new_tag0 = pd.read_json('data/new_tag/new_tag0.json')
new_tag0.reset_index(inplace=True)

new_tag1 = pd.read_json('data/new_tag/new_tag1.json')
new_tag1.reset_index(inplace=True)

new_tag2 = pd.read_json('data/new_tag/new_tag2.json')
new_tag2.reset_index(inplace=True)

new_tag3 = pd.read_json('data/new_tag/new_tag3.json')
new_tag3.reset_index(inplace=True)

In [89]:
train = pd.read_json('data/train.json')
train.reset_index(inplace=True)

In [91]:
new_train0 = pd.merge(train, new_tag0, on='index')
new_train0.drop('index',axis=1,inplace=True)
new_train0 = new_train0[['tags','id','안지움','plylst_title','songs','like_cnt','updt_date']]
new_train0.to_json('data/new_tag/new_train0.json', orient = 'records')

In [96]:
new_train1 = pd.merge(train, new_tag1, on='index')
new_train1.drop('index',axis=1,inplace=True)
new_train1 = new_train1[['tags','id','한개지움','plylst_title','songs','like_cnt','updt_date']]
new_train1.to_json('data/new_tag/new_train1.json', orient = 'records')

In [97]:
new_train2 = pd.merge(train, new_tag2, on='index')
new_train2.drop('index',axis=1,inplace=True)
new_train2 = new_train2[['tags','id','두개지움','plylst_title','songs','like_cnt','updt_date']]
new_train2.to_json('data/new_tag/new_train2.json', orient = 'records')

In [100]:
new_train3 = pd.merge(train, new_tag3, on='index')
new_train3.drop('index',axis=1,inplace=True)
new_train3 = new_train3[['tags','id','세개지움','plylst_title','songs','like_cnt','updt_date']]
new_train3.to_json('data/new_tag/new_train3.json', orient = 'records')